Here’s a Python guide to building an image detection model for friendship sloops. This example uses the TensorFlow library with a simple CNN approach and OpenCV for image preprocessing. You may need to adapt based on your dataset and model requirements.

In [ ]:
%pip install tensorflow
%pip install opencv-python

In [ ]:
from random import randrange
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import cv2
import os
import pprint
from pathlib import Path
from dotenv import load_dotenv

# Suppress only the single warning from urllib3.
import urllib3
urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

# Load environment variables from the .env file (if present)
load_dotenv()

# Define the config class
class CFG:

    # Define the directory to store the images
    DATA_DIR = Path(os.environ['DATA_DIR'])
    
    # Set the number of batchs for processing
    BATCH_SIZE = int(os.environ['BATCH_SIZE'])

    # Epocs for model training
    NUM_EPOCHS = int(os.environ['NUM_EPOCHS'])
    
    MODEL_PATH = Path(os.environ['MODEL_PATH'])
    
pprint.pprint(CFG.__dict__)

## 1. Import Libraries and Prepare Data

In [ ]:
# Set up directories
data_dir = CFG.DATA_DIR
img_size = 224 # 128  # Adjust based on image size
batch_size = CFG.BATCH_SIZE

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split for training/validation
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Load ResNet50 with pretrained weights and exclude top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

print(f'Made: {x}')


## 2. Show sample of the loaded images

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Get a list of image file paths
dir = CFG.DATA_DIR / 'train/friendship_sloop'

image_files = [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith('.jpg') or f.endswith('.png')]

# Display a 3x3 grid of images
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for img_path, ax in zip(image_files[:9], axes):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(os.path.basename(img_path))

plt.tight_layout()
plt.show()

## 3. Build the Model


In [ ]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


## 4. Train the Model


In [ ]:
history = model.fit(
    train_gen,
    epochs=CFG.NUM_EPOCHS,
    validation_data=val_gen
)
print(f"Training completed.")

## 5. Evaluate and Save the Model

In [ ]:
loss, accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Loss: {loss}")

# Create the base directory if it doesn't exist
base_dir = CFG.MODEL_PATH.parent
os.makedirs(base_dir, exist_ok=True)

# Save model
outfile = f'{CFG.MODEL_PATH}.keras'
model.save(outfile)
print(f'saved { os.path.getsize(outfile) } bytes to {outfile}')


## 6. Model Testing on New Images

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img / 255.0
    img = img.reshape(-1, img_size, img_size, 3)
    return img

# Predict
# Directory containing test images
test_image_dir = 'images/prediction'

# Iterate over all images in the directory
for filename in os.listdir(test_image_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        test_image_path = os.path.join(test_image_dir, filename)
        
        # Show the test image
        plt.imshow(Image.open(test_image_path))
        plt.axis('off')
        plt.show()

        # Preprocess and predict
        img = preprocess_image(test_image_path)
        prediction = model.predict(img)

        print(f"Prediction for {filename}: {prediction}")
        if prediction[0][0] > 0.5:
            print("Friendship Sloop detected")
        else:
            print("No Friendship Sloop detected")

